In [1]:
import pandas as pd
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import peakutils as pkuts
from scipy import fftpack
from scipy.interpolate import interp1d
from scipy.fft import fft, fftfreq
# from matplotlib.offsetbox import AnchoredOffsetbox
import datetime as datetime
from datetime import datetime, timedelta
import os
import ctypes
import seaborn as sns
import glob
# import imufusion
import sys
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None  # default='warn'

In [7]:
lengthFlag = 0

def zipper_data(dat_sleep):
    """
    This function adds a new line to a DataFrame by dropping certain columns, renaming columns, and concatenating DataFrames.
    @param dat_sleep - the original DataFrame
    @return a new DataFrame with added lines
    """
    df1 = dat_sleep.drop(columns={'XL_X2','XL_Y2','XL_Z2','G_X2','G_Y2','G_Z2'})
    df2 = dat_sleep.drop(columns={'XL_X','XL_Y','XL_Z','G_X','G_Y','G_Z'})
    df2 = df2.rename(columns={'XL_X2':'XL_X','XL_Y2':'XL_Y','XL_Z2':'XL_Z','G_X2':'G_X','G_Y2':'G_Y','G_Z2':'G_Z'})
    lenOfFile = len(dat_sleep)

    df = pd.DataFrame(columns=["XL_X", "XL_Y", "XL_Z", "G_X", "G_Y", "G_Z", "Temperature", "Voltage", "Packet"])
    for i in tqdm(range(0, lenOfFile)):
        df = pd.concat([df, pd.DataFrame([df1.iloc[i]])])
        df = pd.concat([df, pd.DataFrame([df2.iloc[i]])])

    return df

def extrapolate_time(data, sampling_frequency, num_extrapolated_points):
    """
    This function extrapolates time values based on the sampling frequency provided.
    @param data - The input data, which can be a pandas DataFrame or a list.
    @param sampling_frequency - The frequency at which the data is sampled.
    @param num_extrapolated_points - The number of points to extrapolate.
    @return A list of extrapolated timestamps.
    """
    """Extrapolates time values based on the sampling frequency."""

    # Convert data to a pandas DataFrame if it's not already
    if not isinstance(data, pd.DataFrame):
        data = pd.DataFrame(data)

    # Calculate the time interval between samples
    time_interval = 1 / sampling_frequency

    # Get the last timestamp in the data
    first_timestamp = data.index[0]

    # Generate extrapolated timestamps
    extrapolated_timestamps = [first_timestamp + timedelta(seconds=(i + 1) * time_interval) 
                               for i in range(num_extrapolated_points)]

    return extrapolated_timestamps

def data_processing(dat_lo, hexdat_lo):
    """
    Process the data from the input dat_lo and hexdat_lo to create a new dataframe dat_sleep.
    @param dat_lo - the input data frame
    @param hexdat_lo - the hex data frame
    @param date - the date in the format MMDDYYYY
    @return dat_sleep - the processed data frame
    """
    # print(dat_lo)
    # print(hexdat_lo)
    # Refer to the website: https://github.com/adafruit/Adafruit_LSM6DS/blob/master/Adafruit_LSM6DS.cpp
    # Depends on the configuration of IMU on CCS
    # dat_lo = dat_lo[0:10]
    dat_lo = dat_lo.assign(Temperature = np.nan, Voltage = np.nan) # adding 2 new columns temp and voltage

    # scale ranges for converting hexadecimal data to physical units
    XL_scale_range_2g = 0.061
    XL_scale_range_4g = 0.122
    G_scale_range_1000dps = 35
    G_scale_range_2000dps = 70
    print("Added temp and voltage columns")
    lengthFlag = 0

    print("Starting to convert hex to float")
    # Convert hex to float of acceleration and angular velocity for local data
    for k, ele in tqdm(enumerate(hexdat_lo)): # k: count; ele: item; local data
        # print(f'index: {k}')
        # print(ele)
        if len(ele) == 12: # Only select those that have the correct length (12 for motility)
            lengthFlag = 0
            vlist=[]
            for i in range(0, 6):
                n = i*2 + 2
                value = ''.join(['0x', ele[n-2], ele[n-1]])
                hvalue = int(value, 16)
                convert_value = ctypes.c_int16(hvalue).value
                if i <= 2:
                    vlist.append(round(float(convert_value)*XL_scale_range_4g/1000, 2))
                else:
                    vlist.append(round(float(convert_value)*G_scale_range_1000dps/1000, 2))
            # dat_lo.loc[k, 'Message'] = vlist
            dat_lo.Message[k] = vlist

        elif len(ele) == 13: # Only select those that have the correct length (13 for additional device motility)
            vlist=[]
            lengthFlag = 0
            for i in range(0, len(hexdat_lo)):
                hexdat_lo[i] = hexdat_lo[i][1:13] # To remove '00' in the front

            for i in range(0, 6):
                n = i*2 + 2
                value = ''.join(['0x', ele[n-2], ele[n-1]])
                hvalue = int(value, 16)
                convert_value = ctypes.c_int16(hvalue).value
                if i <= 2:
                    vlist.append(round(float(convert_value)*XL_scale_range_4g/1000, 2))
                else:
                    vlist.append(round(float(convert_value)*G_scale_range_1000dps/1000, 2))
            # dat_lo.loc[k, 'Message'] = vlist
            dat_lo.Message[k] = vlist

        elif len(ele) == 24: # Only select those that have the correct length (12 for 2 * motility)
            lengthFlag = 1
            vlist=[]
            first_ele = ele[0:12]
            second_ele = ele[12:24]
            # print(first_ele)

            for i in range(0, 6):
                n = i*2 + 2
                value = ''.join(['0x', first_ele[n-2], first_ele[n-1]])
                hvalue = int(value, 16)
                convert_value = ctypes.c_int16(hvalue).value
                if i <= 2:
                    vlist.append(round(float(convert_value)*XL_scale_range_4g/1000, 2))
                else:
                    vlist.append(round(float(convert_value)*G_scale_range_1000dps/1000, 2))

            for i in range(0, 6):
                n = i*2 + 2
                value = ''.join(['0x', second_ele[n-2], second_ele[n-1]])
                hvalue = int(value, 16)
                convert_value = ctypes.c_int16(hvalue).value
                if i <= 2:
                    vlist.append(round(float(convert_value)*XL_scale_range_4g/1000, 2))
                else:
                    vlist.append(round(float(convert_value)*G_scale_range_1000dps/1000, 2))
            
            # print(f'msg: {vlist}')
            dat_lo.Message[k] = vlist
            
        elif len(ele) == 4: # length 4 is for voltage/temperature
            dat_lo.Message[k] = 'NaN'
            for i in range(0, 2):
                n = i*2 + 2
                value = ''.join(['0x', ele[n-2], ele[n-1]])
                hvalue = int(value, 16)
                convert_value = ctypes.c_int16(hvalue).value
                if i == 0:
                #   vlist.append(float((convert_value/1000)))
                    dat_lo.Voltage[k] = float((convert_value/1000))
                else:
                #  vlist.append(float(convert_value))
                    dat_lo.Temperature[k] = float(convert_value)

        else:
            #print('Line Error!'+str(k))
            #print(ele)
            vlist = [] #add empty list on error
    print("finished converting hex to float")

    if lengthFlag == 0:
        print("Length flag is 0")
        dat_lo[['XL_X', 'XL_Y', 'XL_Z', 'G_X', 'G_Y', 'G_Z']] = np.nan
        dat_lo = dat_lo[dat_lo['Message'].notna()]
        dat_lo[['XL_X', 'XL_Y', 'XL_Z', 'G_X', 'G_Y', 'G_Z']] = pd.DataFrame(dat_lo['Message'].tolist(), index=dat_lo.index)
        dat_sleep = pd.DataFrame(dat_lo, columns=["XL_X", "XL_Y", "XL_Z", "G_X", "G_Y", "G_Z", "Temperature", "Voltage", "Packet"])
        dat_sleep.loc[:,['Temperature', 'Voltage']] = dat_sleep.loc[:,['Temperature', 'Voltage']].ffill().bfill()
        dat_sleep = dat_sleep[dat_sleep['XL_X'].notna()]
        # print(dat_sleep)

    elif lengthFlag == 1:
        print("Length flag is 1")
        print(dat_lo)
        print(dat_lo.Message)
        dat_lo.loc[:,['Temperature', 'Voltage']] = dat_lo.loc[:,['Temperature', 'Voltage']].ffill().bfill()
        dat_lo['Message'] = dat_lo['Message'].apply(lambda x: x if x != 'NaN' else np.nan)
        dat_lo = dat_lo[dat_lo['Message'].notna()]
        print(dat_lo)
        dat_lo[['XL_X', 'XL_Y', 'XL_Z', 'G_X', 'G_Y', 'G_Z', 'XL_X2', 'XL_Y2', 'XL_Z2', 'G_X2', 'G_Y2', 'G_Z2']] = np.nan
        print(dat_lo)
        dat_lo.loc[:,['Temperature', 'Voltage']] = dat_lo.loc[:,['Temperature', 'Voltage']].ffill().bfill()
        dat_lo = dat_lo.dropna(subset=['Message'])
        dat_lo[['XL_X', 'XL_Y', 'XL_Z', 'G_X', 'G_Y', 'G_Z', 'XL_X2', 'XL_Y2', 'XL_Z2', 'G_X2', 'G_Y2', 'G_Z2']] = pd.DataFrame(dat_lo['Message'].tolist(), index=dat_lo.index)
        dat_lo = dat_lo[dat_lo['Message'].notna()]
        dat_sleep = pd.DataFrame(dat_lo, columns=["XL_X", "XL_Y", "XL_Z", "G_X", "G_Y", "G_Z", 'XL_X2', 'XL_Y2', 'XL_Z2', 'G_X2', 'G_Y2', 'G_Z2', "Temperature", "Voltage", "Packet"])
        print(dat_sleep)
        dat_sleep = zipper_data(dat_sleep)
    return dat_sleep




def detect_sample_frequency(sample):
    """
    Detect the frequency of the given sample data.
    @param sample - The sample data for which frequency needs to be detected.
    @return The inferred frequency of the sample data.
    """
    # s = sample.Time
    # print(s)
    # fs = sample.head(n=100)
    fs = np.diff(sample['Time'].values).min()
    print(fs)
    return fs

def convert_raw_data(file):
    """
    file (input, python list):  list of txt file names with the IMU data
    dat (output, pd DataFrame): table with columns Time, MISC, Packet, Message, RSSI
    hexdat (output, pd DataFrame): IMU binary data convered to hexadecimal 
    date (otuput, datetime): date in the format MMDDYYYY

    Time: In format HH:MM:SS.MS
    MISC: Type of data, f is for voltage, e for temperature and 6 for motility ??
    Packet: Gives the packet number
    Message: This is in bits
    RSSI: Received Signal Strength Indicator, measure that represents the relative quality level of a Bluetooth signal received on a device in decibels-milliwatts (dBm).
    """

    # arranging the data in a readable format as a pd variable
    header = 0
    dat = pd.read_csv(file, header = header, dtype = str, delimiter = '|', names = ['Time','MISC','Packet','Message','RSSI'])
    dat = dat[~dat.RSSI.str.contains('error')] # excludes any error messages when the device is out of range
    # dat = dat[dat.MISC.str.contains('f|e|6')] # To keep voltage, temperature and motility data only
    dat = dat.reset_index(drop=True)
    dat['timedelta'] = pd.to_timedelta(dat['Time'])
    # print(dat)

    # test
    # dat = dat[683333:1514188]
    # dat = dat[dat.MISC == ' 0e '] # local
    # print(dat)
    
    # decoding the message from bits to hex
    dat_col = dat.Message
    dat_col = dat_col.str.strip()
    hexdat = dat_col.str.split(' ') 
    # print(hexdat)

    return dat, hexdat
    # return dat, hexdat, fs

def create_folder_if_doesnt_exist(folder):
    """
    Create a folder if it doesn't exist.
    @param folder - The folder to create.
    """
    if not os.path.exists(folder):
        os.makedirs(folder)

def output_single_file_of_unprocessed_data(filelist, start_date):
    if filelist == []:
        print('No files found')
        return
    
    elif len(filelist) == 1:
        data, hexdata = convert_raw_data(filelist[0])
    
    else:
        appended_data = []
        appended_hexdata = []
        for file in sorted(filelist):
            data, hexdata = convert_raw_data(file)
            print(data)
            appended_data.append(data)
            appended_hexdata.append(hexdata)

        data = pd.concat(appended_data).reset_index(drop=True)
        hexdata = pd.concat(appended_hexdata).reset_index(drop=True)

    # data = data[0:10]
    # hexdata = hexdata[0:10]
    print(data)
    data['timediff'] = data['timedelta'].diff()
    data['timediff'] = data['timediff'].dt.total_seconds()*1000
    data['NewDay'] = 0 # create a new column for the day
    day = 0 # reset the day counter

    print("Data with daytime difference being converted")
    for i, row in tqdm(data.iterrows(), total=len(data)):
        if row['timediff']< 0:
            day += 1
            data.loc[i, 'NewDay'] = day
        else:
            data.loc[i, 'NewDay'] = day

    print(f'Number of days in dataset: {day}')
    print(data)
    
    data['Date'] = start_date + pd.to_timedelta(data['NewDay'], unit='d')
    data['Timestamp'] = data['Date'] + data['timedelta']
    data.set_index('Timestamp', inplace=True)
    # data.index = data['Timestamp']
    fs = data['timediff'].median()/1000
    data.drop(columns=['Time', 'timedelta', 'timediff', 'NewDay', 'Date'], inplace=True)
    print(data)
    return data, hexdata, fs


In [9]:
pd.options.mode.chained_assignment = None




##############################


# --- PARAMETERS ---
study_date   = '07022025'
data_folder  = r'C:\Users\krisf\OneDrive\Desktop\UROP SU25\Ceara sleep detection\Modified Data'
raw_dir      = os.path.join(data_folder, 'raw', study_date)
output_dir   = os.path.join(data_folder, 'processed'+study_date)

# --- FIND ALL .TXT FILES FOR THIS STUDY DATE ---
filelist = glob.glob(os.path.join(raw_dir, '*.txt'))

# --- SPLIT RAW_DIR INTO FOLDER & “FILENAME” (if still needed) ---
input_folder, input_filename = os.path.split(raw_dir)

# --- PARSE THE STUDY DATE ---
# Note: format is YYYYMMDD
start_date = datetime.strptime(study_date, '%m%d%Y')

chunksize = 10000

# --- PROCESS & WRITE OUT ---
data, hexdata, fs = output_single_file_of_unprocessed_data(filelist, start_date)



# print(f'Sample frequency estimate of dataset: {fs}')

# # Prepare the per-date subfolder under processed/
# appended_folder = os.path.join(output_dir, 'appendedNotProcessed', study_date)
# os.makedirs(appended_folder, exist_ok=True)

# # Write CSVs (pandas accepts Path-like or string paths)
# data.to_csv(os.path.join(appended_folder, f'{study_date}_data.csv'), index=False)
# hexdata.to_csv(os.path.join(appended_folder, f'{study_date}_hexdata.csv'), index=False)

                 Time  MISC   Packet                                 Message  \
0       08:35:26.300    0e    54991    02 12 ff d2 e0 50 00 0f ff f7 ff ef     
1       08:35:26.300    0e    54992    02 14 ff d8 e0 75 00 0f 00 07 ff f0     
2       08:35:26.300    0e    54993    02 18 ff cb e0 8f 00 0f 00 0d ff f0     
3       08:35:26.300    0e    54994    02 11 ff da e0 95 00 0f 00 0d ff ef     
4       08:35:26.300    0e    54995    02 24 ff e0 e0 8c 00 10 00 0a ff f0     
...               ...   ...      ...                                     ...   
375787  09:35:45.908    0e    45415    10 d6 ed 03 13 72 ff fb ff f7 00 38     
375788  09:35:45.908    0e    45416    10 e2 ec e3 13 2e 00 07 00 32 00 6b     
375789  09:35:45.908    0e    45417    10 a9 ec da 13 4b 00 00 00 42 00 7f     
375790  09:35:45.908    0e    45418    10 67 ec c1 13 8d ff f7 00 35 00 54     
375791  09:35:45.952    0e    45419    10 68 ec d1 13 ec ff fc 00 1e ff e7     

         RSSI              timedelta  


100%|████████████████████████████████████████████████████████████████████████| 375792/375792 [03:55<00:00, 1597.48it/s]


Number of days in dataset: 0
                 Time  MISC   Packet                                 Message  \
0       08:35:26.300    0e    54991    02 12 ff d2 e0 50 00 0f ff f7 ff ef     
1       08:35:26.300    0e    54992    02 14 ff d8 e0 75 00 0f 00 07 ff f0     
2       08:35:26.300    0e    54993    02 18 ff cb e0 8f 00 0f 00 0d ff f0     
3       08:35:26.300    0e    54994    02 11 ff da e0 95 00 0f 00 0d ff ef     
4       08:35:26.300    0e    54995    02 24 ff e0 e0 8c 00 10 00 0a ff f0     
...               ...   ...      ...                                     ...   
375787  09:35:45.908    0e    45415    10 d6 ed 03 13 72 ff fb ff f7 00 38     
375788  09:35:45.908    0e    45416    10 e2 ec e3 13 2e 00 07 00 32 00 6b     
375789  09:35:45.908    0e    45417    10 a9 ec da 13 4b 00 00 00 42 00 7f     
375790  09:35:45.908    0e    45418    10 67 ec c1 13 8d ff f7 00 35 00 54     
375791  09:35:45.952    0e    45419    10 68 ec d1 13 ec ff fc 00 1e ff e7     

         R

In [12]:
data = data_processing(data, hexdata)
data.to_csv(f'{output_dir}/{study_date}_imuprocessed.csv')

data
###############################








############ PREPROCESSING ##############
# folder date
# study_date = '20250407'
# data_folder = '../data/IMU/'
# directory = f'{data_folder}raw/{study_date}/'
# filelist = glob.glob(directory+'*.txt')
# input_folder, input_filename = os.path.split(directory)
# output_folder = f'{data_folder}processed/'
# start_date = datetime.strptime(study_date, '%Y%m%d') #which takes in "MMDDYYYY" like only US people write date order
# chunksize = 10000


# ### WRITE STUDY OUTPUT FILES TO A SINGLE CSV AND SETUP FOR PROCESSING ###
# data, hexdata, fs = output_single_file_of_unprocessed_data(filelist, start_date)
# # print(f'Sample frequency estimate of dataset: {fs}')

# ### WRITE TO CSV IF APPLICABLE ###
# create_folder_if_doesnt_exist(f'{output_folder}appendedNotProcessed/{study_date}/')
# data.to_csv(f'{output_folder}appendedNotProcessed/{study_date}/{study_date}_data.csv')
# hexdata.to_csv(f'{output_folder}appendedNotProcessed/{study_date}/{study_date}_hexdata.csv')
# # TODO: add in analysis for multiple sensors in one file

# #   ## WRITE TO CSV ###
# # input_folder, input_filename = os.path.split(file)
# # process the message as a function IMU data
# create_folder_if_doesnt_exist(f'{output_folder}{study_date}/')

# ## BATCH PROCESS CSV ###
# # for datChunk in pd.read_csv(f'{output_folder}appendedNotProcessed/{study_date}/{study_date}_data.csv', index_col='datetime', chunksize=chunksize):
# #     datChunk.reset_index(inplace=True)
# #     dat_col = datChunk.Message
# #     dat_col = dat_col.str.strip()
# #     hexdata = dat_col.str.split(' ')
# # data.to_csv(f'{output_folder}{study_date}/{study_date}_imuprocessed.csv', mode='a', header=False, index=False)

# ## NOT BATCH PROCESS CSV ###
# data = data_processing(data, hexdata)
# data.to_csv(f'{output_folder}{study_date}/{study_date}_imuprocessed.csv')

############ ANALYSIS ##############
# cols = ['Timestamp', 'XL_X', 'XL_Y', 'XL_Z', 'G_X', 'G_Y', 'G_Z', "Temperature", "Voltage", "Packet"]
# data = pd.read_csv(f'{output_folder}{study_date}/{study_date}_imuprocessed.csv')
# data.rename(columns={data.columns[0]: 'Timestamp'}, inplace=True)
# data['Timestamp'] = pd.to_datetime(data['Timestamp'])
# data.set_index('Timestamp', inplace=True)
# lowcut_hr = 2
# highcut_hr = 22
# i=100
# data = calculate_euler_angles(data, fs)
# print(data)
# plot_euler_angles(data, i)
# upper_thresh, lower_thresh, piecewise_linear_peaks_outl_rm, piecewise_linear_valleys_outl_rm, angle_w, mov_avg, time_w, peaks, valleys = peak_detection(data, fs)
# plot_peak_detection(upper_thresh, lower_thresh, piecewise_linear_peaks_outl_rm, piecewise_linear_valleys_outl_rm, angle_w, mov_avg, time_w, peaks, valleys, i)
# print(data)



Added temp and voltage columns
Starting to convert hex to float


375792it [06:15, 1001.42it/s]


finished converting hex to float
Length flag is 0


,XL_X,XL_Y,XL_Z,G_X,G_Y,G_Z,Temperature,Voltage,Packet
Timestamp,,,,,,,,,
2025-07-02 08:35:26.300,0.06,-0.01,-0.99,0.53,-0.32,-0.59,28.0,2.746,54991
2025-07-02 08:35:26.300,0.06,-0.0,-0.99,0.53,0.24,-0.56,28.0,2.746,54992
2025-07-02 08:35:26.300,0.07,-0.01,-0.98,0.53,0.46,-0.56,28.0,2.746,54993
2025-07-02 08:35:26.300,0.06,-0.0,-0.98,0.53,0.46,-0.59,28.0,2.746,54994
2025-07-02 08:35:26.300,0.07,-0.0,-0.98,0.56,0.35,-0.56,28.0,2.746,54995
...,...,...,...,...,...,...,...,...,...
2025-07-02 09:35:45.908,0.53,-0.59,0.61,-0.17,-0.32,1.96,41.0,2.976,45415
2025-07-02 09:35:45.908,0.53,-0.6,0.6,0.24,1.75,3.75,41.0,2.976,45416
2025-07-02 09:35:45.908,0.52,-0.6,0.6,0.00,2.31,4.45,41.0,2.976,45417
